# Stage 2: Image Feature Analysis

**Rui Filipe Martins Monteiro (R20170796) | MSc in Data Science and Advanced Analytics**

<br>

This notebook loads the intermediate datasets and performs classification with M4GP. Significance tests are performed on the output to determine statistical significance.

[ellyn](https://cavalab.org/ellyn/), a genetic programming system for regression, is one of the most important Python libraries, as it is needed to apply M4GP. *Note:* As of February 2022, ellyn is not compatible with Windows, so this needs to be run on Linux.

<br>

This code is heavily inspired by: Jonathan Janke (https://github.com/novajon/classy-conv-features)

Code changed and improved by: Rui Monteiro

In [1]:
# Imports
import glob
import numpy as np
import pandas as pd
import copy

from ellyn import ellyn
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    cohen_kappa_score
)
from sklearn.model_selection import GridSearchCV, train_test_split, KFold

# import keras
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Dropout

import os
import datetime
import time
import pickle
import statistics
# from scipy import stats

# from scipy.stats import ttest_ind_from_stats
# import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
def iterate_over_datasets(datasets, function):
    """Iterate over datasets and apply function on each one

    Parameters
    ----------
    datasets : list
        list of datasets that function should be applied on
    function : function
        function that should be applied on each dataset
    """
    ret_data = []
    
    for data in datasets:
        n_data = function(data)
        ret_data.append(n_data)
        
    return ret_data

In [3]:
def split_input_target (data):
    """Save the target data separately from the input data

    Parameters
    ----------
    data : list
        list containing input and output data together
    """
    target = []
    inp = []
    
    for d in data:
        target.append(d[1][0])
        inp.append(d[0])
        
    return [np.array(inp), target]

In [4]:
def reindex_classes(data):
    """Reindexing classes to be continuous and start at 0

    Parameters
    ----------
    data : list
        dataset to reindex
    """
    for ind, ds in enumerate(data):
        ds[1] = [x - min (ds[1]) for x in ds[1]]
        data[ind] = ds
        
    return data

In [5]:
def get_top_n_accuracy(input_target, input_data, n=5):
    """Get top n accuracy from input data (predictions)

    Parameters
    ----------
    input_target : list
        array of correct targets
    input_data : list
        array of predictions
    n : int
        n value for top-n-accuracy
    """
    count = 0
    inp_data = input_data.copy()
    
    for ind, pred in enumerate(inp_data):
        max_classes = []
        
        for i in range(n):
            max_classes.append(np.argmax(pred))
            pred[np.argmax(pred)]=-1
            
        if input_target[ind] in max_classes: 
            count += 1
            
    return count/len(input_target)

In [6]:
def evaluate(model, data):
    """Method to evaluate a model based on several metrics:
        - top-1-accuracy ("accuracy")
        - top-2-accuracy
        - top-5-accuracy
        - top-10-accuracy
        - top-20-accuracy
        - confusion matrix
        - precision
        - recall
        - f1-score
        - cohen's kappa

    Parameters
    ----------
    model : keras/sklearn model (needs predict function)
        dataset to reindex

    data : list
        dataset to evaluate
    """
    y_pred = model.predict(data[0])
    
    try: y_proba = model.predict_proba(data[0])
    except: y_proba = model.predict(data[0])
    
    try: predictions = [round(value) for value in y_pred]
    except: predictions = [np.argmax(value) for value in y_pred]
    
    # Evaluate predictions
    d = {}
    d["accuracy"] = accuracy_score(data[1], predictions)
    print('y_proba, is it a problem?:', y_proba) # DEL LATER!
    #d["top-2-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=2)
    #d["top-5-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=5)
    #d["top-10-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=10)
    #d["top-20-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=20)
    d["confusion matrix"] = confusion_matrix(data[1], predictions)
    d["precision"] = precision_score(data[1], predictions, average='macro')
    d["recall"] = recall_score(data[1], predictions, average='macro')
    d["f1-score"] = f1_score(data[1], predictions, average='macro')
    d["cohen's kappa"] = cohen_kappa_score(data[1], predictions)
    
    return d

In [7]:
def get_measures(measure, data_type="data", measure_name="Accuracy"):
    """Get string representation of measures

    Parameters
    ----------
    measure : float
        value to print
    data_type : string
        usually test or training data
    measure_name : string
        measure that is applied, e.g. 'Accuracy'
    """
    return "%s in %s: %.2f" % (measure_name, data_type, measure)

In [8]:
def print_measures(evaluation, t):
    """Print measures on screen

    Parameters
    ----------
    evaluation : dict
        key value pairs with accuracy measure : value
    t : string
        Data type, e.g. "Test" or "Train"
    """
    for key in evaluation.keys():
        if key != "confusion matrix":
            print(get_measures(evaluation[key], t, key))
            
        else:
            print(key)
            print(evaluation[key])

In [9]:
def write_to_file(train_eval, test_eval, model_stats, model_key, model_name, output_file_name):
    """Save the results to a CSV file

    Parameters
    ----------
    train_eval : dict
        key value pairs of "evaluation metric name : evaluation metric value" on train data
    test_eval : dict
        key value pairs of "evaluation metric name : evaluation metric value" on test data
    model_stats : dict
        key value pair of "train"/"test" : model statistics (e.g., mean, std. dev, ...)
    model_key : string
        name of the model that was applied
    model_name : string
        name of the model that was used to create the intermediate dataset, e.g., VGG-16
    output_file_name : string
        name of the file to write to
    """
    file_stamp = curr_path.split("/")[-2]
    train_size = str(len(train_data[0][0]))
    test_size = str(len(test_data[0][0]))
    dataset_dim = str(np.max(train_data[0][1] + test_data[0][1]) + 1)
    
    with open(output_file_name + '_train.csv', 'a') as f:
        print (output_file_name + '_train.csv')
        f.write("\n" + model_name + ',' + file_stamp + ',' + train_size + ',' + dataset_dim + ',' + model_key + ",")
        f.write(','.join([str(train_eval[x]) for x in train_eval.keys() if x!="confusion matrix"]) + ",")
        f.write(','.join([str(x) for x in model_stats["train"].values()]))
    
    with open(output_file_name + '_test.csv', 'a') as f:
        f.write("\n" + model_name + ',' + file_stamp + ',' + test_size + ',' + dataset_dim + ',' + model_key + ",")
        f.write(','.join([str(test_eval[x]) for x in test_eval.keys() if x!="confusion matrix"]) + ",")    
        f.write(','.join([str(x) for x in model_stats["test"].values()]))

In [10]:
def create_data_files(ind,prefix):
    """Create initial files to write to

    Parameters
    ----------
    ind : int
        index of dataset
    prefix : string
        prefix to give to file name
    """
    path = "/home/ruifi/thesis/outputs/benchmark_results/" + prefix + "_" + dataset_name + "/"
    output_file_name = path + prefix + "_" + str(ind)
    
    if not os.path.exists(path):
        os.makedirs(path)
        
    with open(output_file_name + '_train.csv', 'w') as f:
        f.write('Dataset,Dataset Stamp,Dataset Size,# Dataset Classes,Model Architecture' 
                + ',' + "accuracy" + ',' + "top-2-accuracy" + ',' + "top-5-accuracy" + ',' 
                + "top-10-accuracy" + ',' + "top-20-accuracy" + ',' + "precision" + ',' 
                + "recall" + ',' + "f1-score" + ',' + "cohen's kappa" + ',' 
                + "Mean Accuracy over cross validation" + ',' + "mean standard deviation over cross validation" 
                + ',' + "number observations over cross validation")
        
    with open(output_file_name + '_test.csv', 'w') as f:
        f.write('Dataset,Dataset Stamp,Dataset Size,# Dataset Classes,Model Architecture' 
                + ',' + "accuracy" + ',' + "top-2-accuracy" + ',' + "top-5-accuracy" + ',' 
                + "top-10-accuracy" + ',' + "top-20-accuracy" + ',' + "precision" 
                + ',' + "recall" + ',' + "f1-score" + ',' + "cohen's kappa" + ',' 
                + "Mean Accuracy over cross validation" + ',' + "mean standard deviation over cross validation" 
                + ',' + "number observations over cross validation")
        
    return output_file_name

In [11]:
def get_model_stats(cv_results, best_params, folds):
    """Method to get model statistics from training run, like mean, standard deviation and number of observations.
    This helps calculating statistical significance in the end.

    Parameters
    ----------
    cv_results : dict
        dictionary with results from cross-validation, e.g. train/test mean/std. dev. 
    best_params: list:
        best parameter configuration
    folds : number of cross validation folds
    """
    truth = [param == best_params for param in cv_results['params']]
            
    curr_model_stats = {}
    curr_model_stats["train"] = {}
    curr_model_stats["train"]["mean"] = cv_results['mean_train_score'][truth][0]
    curr_model_stats["train"]["std_dev"] = cv_results['std_train_score'][truth][0]
    curr_model_stats["train"]["observations"] = folds
    curr_model_stats["test"] = {}
    curr_model_stats["test"]["mean"] = cv_results['mean_test_score'][truth][0]
    curr_model_stats["test"]["std_dev"] = cv_results['std_test_score'][truth][0]
    curr_model_stats["test"]["observations"] = folds
    
    print("Mean: {}".format(cv_results['mean_test_score'][truth]))
    print("Standard deviation: {}".format(cv_results['std_test_score'][truth]))
    print("Number of observations: {}".format(folds))
    
    return curr_model_stats

In [12]:
# def plot_model_stats(hist):
#     """
#     Plot training curve over time (for neural networks)
#     """
#     plt.plot(hist.history['acc'])
#     plt.plot(hist.history['val_acc'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    
#     # Summarize history for loss
#     plt.plot(hist.history['loss'])
#     plt.plot(hist.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

In [13]:
def grid_search_model(benchmark_models, parameter_candidates, train, test, model_name, 
                      output_file_name, folds = 10, prefix = "00000000"):
    """Perform grid search with model parameters and save results to CSV file and produced models

    Parameters
    ----------
    benchmark_models : dict
        key value pair of model_name:model.
    parameter_candidates : dict
        key value pair of model_name:model parameters.
    train : list
        list with values input data,target that contains the train data
    test : list
        list with values input data,target that contains the test data
    model_name : string
        name of dataset model that was used to create intermediate dataset (e.g., "CIFAR-100")
    output_file_name : string
        name of the output file (that has been previously created)
    folds : int
        number of folds for crossvalidation
    prefix : string
        prefix for output file 
    """
    model_stats = {}
    
    # The following process is done for every model in the list of benchmark_models
    for model_key in benchmark_models.keys():
        model = benchmark_models[model_key]
        print ((" Model: " + str(model_key)+ " ").center(30, '#'))
        start_time = time.time()
        
        # The model could be either from sklearn or from Keras
        # If the following try statement fails, we expect the model to be a Keras model, 
        # because GridSearchCV does not work with Keras models
        try:
            # Initialize grid search for model with specified parameters from parameter_candidates
            #print(model)
            #print(parameter_candidates[model_key])
            #print(folds)
            clf = GridSearchCV(estimator=model, 
                               param_grid=parameter_candidates[model_key], 
                               n_jobs=-1, 
                               cv=folds, 
                               return_train_score=True)
            
            # Perform grid search by fitting each parameter configuration to the training data
            #clf.fit(*train)
            clf.fit(train[0], train[1])
            model = clf.best_estimator_
            print ("Best parameters: {}".format(clf.best_params_))
            
            path = "/home/ruifi/thesis/outputs/benchmark_results/" + prefix + "_" + dataset_name + "/" + model_name + "/"
            print(path)
            
            if not os.path.exists(path):
                os.makedirs(path)
                
            # Save model
            filename = path + model_key + ".sav"

            try:
                pickle.dump(clf, open(filename, 'wb'))
            except:
                print(filename + " could not be created.")
                
            # Get the model statistics from method defined before
            curr_model_stats = get_model_stats(clf.cv_results_, clf.best_params_, folds)
        
        except:
            model = benchmark_models[model_key][model_name]
            
            # Define k-fold with number of folds from function parameters
            kf = KFold(n_splits=folds)
            
            # Keras expects target to be one hot encoded
            target_cat = np.array(to_categorical(train[1]))
            
            # Initialize model stats to be able to perform statistical tests
            cv_results = {}
            cv_results["train"] = []
            cv_results["test"] = []
            curr_model_stats = {
                    "train": {
                        "mean":0,
                        "std_dev":0,
                        "observations":0
                    },
                    "test": {
                        "mean":0,
                        "std_dev":0,
                        "observations":0
                    }
                }
            
            # Save weights to always initialize model with identical weights
            Wsave = model.get_weights()
            
            for epoch in epochs:
                # Perform n-fold split
                for kfold_ind, indices in enumerate(kf.split(train[0])):
                    model.set_weights(Wsave)
                    train_index, test_index = indices
                    X_train, X_test = train[0][train_index], train[0][test_index]
                    y_train, y_test = target_cat[train_index], target_cat[test_index]
                    
                    # Train model
                    history = model.fit(X_train, y_train, epochs = epoch, validation_data = (X_test, y_test))
                    
                    path = "/home/ruifi/thesis/outputs/benchmark_results" + prefix + "_" + dataset_name + "/" + model_name + "/"
                    
                    if not os.path.exists(path):
                        os.makedirs(path)
                    
                    # Save weights of model for future reuse (if necessary)
                    model.save_weights(path + str(kfold_ind) + "_" + model_key + ".h5")
                    print(model_key)
                    
                    # Plot model learning curve
                    plot_model_stats(history)
                    
                    # Evaluate model on training data
                    train_res = model.evaluate(X_train, y=y_train)[1]
                    
                    # Evaluate model on unseen test data
                    test_res = model.evaluate(X_test, y=y_test)[1]
                    
                    # Save results
                    cv_results["train"].append(train_res)
                    cv_results["test"].append(test_res)
                    
                # Overwrite statistics if they have been better
                if curr_model_stats["test"]["mean"] < statistics.mean(cv_results["test"]):
                    curr_model_stats = {
                        "train": {
                            "mean":statistics.mean(cv_results["train"]),
                            "std_dev":statistics.stdev(cv_results["train"]),
                            "observations":len(cv_results["train"])
                        },
                        "test": {
                            "mean":statistics.mean(cv_results["test"]),
                            "std_dev":statistics.stdev(cv_results["test"]),
                            "observations":len(cv_results["test"])
                        }
                    }
                    
        model_stats[model_key] = curr_model_stats           

        # Evaluate final model with evaluate function from above (various metrics)
        train_eval = evaluate(model, train)
        test_eval = evaluate(model, test)
        
        # Print evaluation metrics
        print_measures(train_eval, "Train")
        print_measures(test_eval, "Test")
        print("Total Time: {}".format(time.time() - start_time))
        
        # Save evaluation metrics in file
        write_to_file(train_eval, test_eval, curr_model_stats, model_key, model_name, output_file_name)

    return benchmark_models, model_stats

## 1. Loading the datasets

The datasets are pre split between train and test data.

In [14]:
# Check current path
import os

first = os.getcwd()
first

'/home/ruifi/thesis'

In [15]:
# Define the path to the intermediate datasets
path = first + '/outputs/data/cifar10_filtered/intermediate' # For CIFAR-10

In [16]:
# Define the name of the subfolder in path to use
predefined_folder_name = ""

In [17]:
# If no predefined folder name is set, the latest data is used (folder with latest timestamp)
if predefined_folder_name == "":
    folder_list = np.sort(glob.glob(path + "/*"))
    curr_path = folder_list[-1] + "/"
    
else:
    curr_path = path + predefined_folder_name + "/"

In [18]:
# The current path will then consist of the path and the latest timestamp folder
curr_path

'/home/ruifi/thesis/outputs/data/cifar10_filtered/intermediate/20220224_1845/'

In [19]:
# Name of the dataset
dataset_name = curr_path.split("/")[-4].split("_")[0]

In [20]:
dataset_name

'cifar10'

In [21]:
# The model names are all subfolders that are in the folder curr_path
model_names = [fn.split("/")[-1] for fn in glob.glob(curr_path + "*")]

In [22]:
model_names

['CNN_Input_1', 'CNN_Input_2']

In [23]:
# Load .npy files into Python
train_data = []
test_data = []

for fn in model_names:
    train_data_parts = np.load(glob.glob(curr_path + str(fn) + "/train*")[0], allow_pickle=True)
    for tt in glob.glob(curr_path + str(fn) + "/train*")[1:]:
        train_data_parts = np.vstack((train_data_parts,np.load(tt)))
    train_data.append(copy.copy(train_data_parts))
    
    test_data_parts = np.load(glob.glob(curr_path + str(fn) + "/test*")[0], allow_pickle=True)
    for tt in glob.glob(curr_path + str(fn) + "/test*")[1:]:
        test_data_parts = np.vstack((test_data_parts, np.load(tt)))
    test_data.append(copy.copy(test_data_parts))

In [24]:
# We can check the dataset dimensions
for ind, name in enumerate(model_names):
    print(name + " dimension on input data: " + str(train_data[ind][0][0].shape))

CNN_Input_1 dimension on input data: (10,)
CNN_Input_2 dimension on input data: (10,)


In [25]:
# Iterate over train and test data to split the input data from the target for each
train_data = iterate_over_datasets(train_data, split_input_target)
test_data = iterate_over_datasets(test_data, split_input_target)

In [26]:
# Reindex datasets to make their classes continuous?

## 2. Classification Model Benchmark

### M4GP hyperparameters

Check all ellyn parameters here: <br>
https://github.com/cavalab/ellyn/blob/03ebdf07f0bfdca30da1a9a0a0da2989e9b1153c/src/ellyn.py#L42

In [27]:
m4gp_candidates= [
    {'g': [100, 200], 
     'popsize': [50, 100], 
     'selection': ['tournament', 'lexicase'] #,
     #'elitism': [True, False],
     #'tourn_size': [2, 10, 20],
     #'rt_cross': [0.5, 0.8],
     #'rt_mut': [0.2, 0.5]
     }
]

In [28]:
parameter_candidates = {}

parameter_candidates['m4gp'] = m4gp_candidates

In [29]:
# Classifiers in the benchmark

# Parameters needed for M4GP on ellyn
m4gp = ellyn(#verbosity=1, 
             class_m4gp=True, 
             op_list=['v','+','-','*','/','sin','cos','exp','log','sqrt'],
                      #,'=','!','<','<=','>','>=','if-then','if-then-else','&','|'],
             fit_type='F1')

# Experiments:
# op_list=['n','v','+','-','*','/','sin','cos','exp','log','sqrt']
# random_state=42

benchmark_models = {'m4gp': m4gp}

In [30]:
# Look at the benchmark model
benchmark_models

{'m4gp': ellyn(class_m4gp=True, fit_type='F1',
       op_list=['v', '+', '-', '*', '/', 'sin', 'cos', 'exp', 'log', 'sqrt'])}

Run benchmarking function (grid search model) for each dataset:

In [31]:
final_models = []
stats = {}

# Save current time to add as a prefix to file output names
now = datetime.datetime.now()
prefix = now.strftime("%Y%m%d_%H%M")

# Perform benchmark model comparison for each dataset
for ind, _ in enumerate(train_data):
    print (("").center(60, '_'))
    print (("").center(60, '#'))
    print ((" Dataset Index: " + str(ind)+ " ").center(60, '#'))
    print (("").center(60, '#'))
    
    # Create CSV files to write output to and initalize those with table headers
    file_name = create_data_files(ind, prefix)
    
    # Perform grid search with method above
    m,stat = grid_search_model(benchmark_models, 
                               parameter_candidates, 
                               train_data[ind], 
                               test_data[ind], 
                               model_names[ind], 
                               file_name, 
                               10,
                               prefix)
        
    final_models.append(m)
    
    # Save model statistics
    stats[model_names[ind]] = stat

____________________________________________________________
############################################################
##################### Dataset Index: 0 #####################
############################################################
######## Model: m4gp #########
Best parameters: {'g': 200, 'popsize': 100, 'selection': 'lexicase'}
/home/ruifi/thesis/outputs/benchmark_results/20220406_1804_cifar10/CNN_Input_1/
Mean: [0.33528]
Standard deviation: [0.10681443]
Number of observations: 10
y_proba, is it a problem?: [6 4 4 ... 4 1 1]
y_proba, is it a problem?: [1 3 1 ... 5 1 4]
accuracy in Train: 0.53
confusion matrix
[[ 218    1    0   76 4032    0    0  306  200  167]
 [   0 5000    0    0    0    0    0    0    0    0]
 [   0    0 4995    1    4    0    0    0    0    0]
 [ 474   18    0 1046 1586    0    2  436  551  887]
 [ 161    0    0   20 4416    0    0  197  109   97]
 [   0    0    0    0    0 4986   14    0    0    0]
 [   0    6    0    0    0    0 4994    0    0    0

In [ ]:
# Prob 1) I think ellyn doesnt have model.predict_proba. This is a big issue when using 'get_top_n_accuracy'!
# Prob 2) Very low accuracy? The models seem very strange, sometimes they are only decent for one class, 
# and very bad for the others.



# I will print y_proba just to check AND I'll comment the top-n-accuracy for now.
# I can check the previous outputs in my github!

In [ ]:
benchmark_models

In [ ]:
def get_signficance_matrix(stats, base="test", sig_level=0.05, one_sided = False):
    """Return significance matrix of multiple benchmark comparisons

    Parameters
    ----------
    stats : dict
        key value pair of model_name : statistics
        with the structure model_name -> classification_algorithm -> train/test -> mean/std_dev/observations
    base : "string"
        dataset to perform comparison on, e.g. "train", "test" (default: test)
    sig_level : float
        significance level to test for
    one_sided : boolean
        boolean to determine if test should be one sided or not

    returns a dictionary with each being (dataset:significance_matrix as pandas DF), e.g., sig_matrix_dict["InceptionV3"]
    """
    if one_sided:
        sig_level /= 2
        
    sig_matrix_dict = {}
    
    for dataset in stats.keys():
        sig_matrix_dict[dataset] = []
        model_entries = stats[dataset].keys()
        
        for model_key in stats[dataset].keys():
            dim_B = []
            
            for compare_model_key in stats[dataset].keys():
                t_stats = ttest_ind_from_stats(stats[dataset][model_key][base]["mean"], 
                                               stats[dataset][model_key][base]["std_dev"], 
                                               stats[dataset][model_key][base]["observations"], 
                                               stats[dataset][compare_model_key][base]["mean"], 
                                               stats[dataset][compare_model_key][base]["std_dev"], 
                                               stats[dataset][compare_model_key][base]["observations"])
                
                if t_stats[1] < sig_level and t_stats[0] < 0.0:
                    dim_B.append(True)
                else:
                    dim_B.append(False)
                    
            sig_matrix_dict[dataset].append(dim_B)
            
        sig_matrix_dict[dataset] = pd.DataFrame(sig_matrix_dict[dataset],index=model_entries,columns=model_entries)
        
    return sig_matrix_dict

In [ ]:
get_signficance_matrix(stats)